In [1]:
%load_ext autoreload
%autoreload 2

# Description

Problem statement:

A game of bingo is played on a 5x5 bingo card. The card is populated randomly with 25 values in the range 1-75 selected without replacement. Next, 30 numbered balls are drawn in the range 1-75 without replacement. A card position with number matching a numbered ball is called daubed.

A bingo pattern is a collection of bingo card spots. A pattern is hit if, after the 30 balls are drawn, the bingo card is daubed with at least the spots contained on the pattern.

Here are a couple patterns we will consider: the Postage Stamp and 4 Corners.
![Bingo Card](images/bingo.png)

# Problem 1

Since both patterns have 4 daubs, hitting either pattern in 30 balls drawn
has the same probability. What is this probability? Write a monte-carlo simulation for this problem.

## Monte Carlo Simulation:

We set up some imports of libraries that I wrote, along with some parameters for the problem.

In [2]:
from bingo_simulator.bingo_card_generator import generate_bingo_card
from bingo_simulator.random_number_generator import SampleWithoutReplacement
from bingo_simulator.bingo_card import BingoCard
import numpy as np
from bingo_simulator.bingo_game import BingoGame
from bingo_simulator.bingo_patterns import create_postage_pattern

min_value = 1
max_value = 75
num_rows = 5
num_cols = 5
num_to_draw = 30
num_simulations = 100000

sampling_list = list(np.arange(min_value, max_value + 1))
number_generator = SampleWithoutReplacement(sampling_list)



We generate 100000 5x5 bingo cards

In [3]:
bingo_cards: list[BingoCard] = [
    generate_bingo_card(
        nrows=num_rows,
        ncols=num_cols,
        random_number_generator=number_generator,
    )
    for _ in range(num_simulations)
]

We generate 100000 lists of randomly drawn values of size 30

In [4]:
def draw_numbers(number_generator: SampleWithoutReplacement, num_to_draw: int)->list[int]:
    return_list = [number_generator.generate() for _ in range(num_to_draw)]
    number_generator.reset()
    return return_list


In [5]:
sampling_list = list(np.arange(min_value, max_value + 1))
number_generator = SampleWithoutReplacement(sampling_list)
drawn_values_list: list[list[int]] = [draw_numbers(number_generator, num_to_draw) for _ in range(num_simulations)]

The bingo pattern is generated

In [6]:
bingo_pattern = create_postage_pattern(bingo_cards[0])

We assemble all the bingo games 

In [7]:
bingo_games: list[BingoGame] = [
    BingoGame(card, bingo_pattern, drawn_values)
    for card, drawn_values in zip(bingo_cards, drawn_values_list)
]

We count the winners and divide by the total amount of bingo games

In [8]:
successes = [bingo_game.check_winner() for bingo_game in bingo_games]
print(f"{np.sum(successes)/len(bingo_games)*100:.2f}% of the bingo games were won")

2.23% of the bingo games were won


From this, we would expect that the mathematical analysis will show us that about 2% of the games will be won.

## Mathematics

Since each outcome is equally likely, one can simply count the ways that there are successes and divide that by the total number of outcomes. The total number of ways to select 30 numbers from a collection of 75 distinct numbers would be ${75 \choose 30}$. To count the successes, simply fix any of the four in the thirty selected to be the successes, then count the number of ways to select the remaining. There are 71 numbers remaining, and we will select 26 of them. Thus, the total number of successes is ${71 \choose 26}$. Therefore the probability is

$$ \begin{align*} \frac{{71 \choose 26}}{{75 \choose 30}} &= \frac{71!}{45!26!}\frac{30!45!}{75!} \\ &= \frac{30!}{26!}\frac{71!}{75!} \\ &= \frac{(30)(29)(28)(27)}{(75)(74)(73)(72)} \approx 0.02255  \end{align*}$$

So the chance to hit is about 2.255 percent, as suggessted by the simulation.

# Problem 2

What is the probability of hitting the 4 corners pattern given the postage stamp pattern is not hit in 30 balls drawn? 

Write a Monte Carlo simulation to estimate this quantity, then analytically provide the exact probability.

## Monte Carlo Simulation

Some initial configuration, very similar to the previous problem

In [21]:
from bingo_simulator.bingo_card_generator import generate_bingo_card
from bingo_simulator.random_number_generator import SampleWithoutReplacement
from bingo_simulator.bingo_card import BingoCard
import numpy as np
from bingo_simulator.bingo_game import BingoGame
from bingo_simulator.bingo_patterns import (
    create_postage_pattern,
    create_corners_pattern,
)
from dataclasses import dataclass

min_value = 1
max_value = 75
num_rows = 5
num_cols = 5
num_to_draw = 30
num_simulations = 100000

sampling_list = list(np.arange(min_value, max_value + 1))
number_generator = SampleWithoutReplacement(sampling_list)

In [22]:
bingo_cards: list[BingoCard] = [
    generate_bingo_card(
        nrows=num_rows,
        ncols=num_cols,
        random_number_generator=number_generator,
    )
    for _ in range(num_simulations)
]

In [23]:
def draw_numbers(number_generator: SampleWithoutReplacement, num_to_draw: int)->list[int]:
    return_list = [number_generator.generate() for _ in range(num_to_draw)]
    number_generator.reset()
    return return_list

In [24]:
sampling_list = list(np.arange(min_value, max_value + 1))
number_generator = SampleWithoutReplacement(sampling_list)
drawn_values_list: list[list[int]] = [draw_numbers(number_generator, num_to_draw) for _ in range(num_simulations)]

A bit different from the last problem, we'll check both patterns.

In [25]:
postage_pattern = create_postage_pattern(bingo_cards[0])
corners_pattern = create_corners_pattern(bingo_cards[0])

In [26]:
postage_bingo_games: list[BingoGame] = [
    BingoGame(card, postage_pattern, drawn_values)
    for card, drawn_values in zip(bingo_cards, drawn_values_list)
]

In [27]:
corners_bingo_games: list[BingoGame] = [
    BingoGame(card, corners_pattern, drawn_values)
    for card, drawn_values in zip(bingo_cards, drawn_values_list)
]

I introduce a data structure to make the code that follows a little bit more readable

In [28]:
@dataclass
class GamePairResult:
    postage_game_result: bool
    corners_game_result: bool

To get our estimate, we will see what percentage of all the games where the corners bingo game lost has the postage game as a winner.

Let's get all the pairs of results

In [29]:
game_pair_result_list: list[GamePairResult] = [
    GamePairResult(postage_game.check_winner(), corners_game.check_winner())
    for postage_game, corners_game in zip(postage_bingo_games, corners_bingo_games)
]

Get just those that are postage game losses

In [30]:
postage_games_lost_list = [game_pair for game_pair in game_pair_result_list if not game_pair.postage_game_result]

Of the postage losses, see when corners were won

In [31]:
corners_won_in_postage_lost_list = [
    game_pair for game_pair in postage_games_lost_list if game_pair.corners_game_result
]

Print out the percentage

In [32]:
print(
    f"{len(corners_won_in_postage_lost_list)/ len(postage_games_lost_list)*100:.2f} postage stamp pattern games won out of all of the games where the corners pattern lost"
)

2.19 postage stamp pattern games won out of all of the games where the corners pattern lost


## Mathematics

Let $E$ be the event that the four corners is hit, $F$ be the even that postage stamp is hit. Then by definition $$P(E|F^c) = \frac{P(E \cap F^c)}{P(F^c)} $$ where $F^c$ is the complement of F. One can rewrite $ E = (E \cap F) \cup (E \cap F^c)$, and it can be easily be shown that $E \cap F$ and $E \cap F^c$ are disjoint events (these statements are easy to see with a Venn Diagram). Thus $$P(E) = P(E \cap F) + P(E \cap F^c).$$ With the intention of replacing the numerator in the conditional probability expression, we can solve this equation to get $$P(E \cap F^c) = P(E) - P(E \cap F).$$ Now the conditional probability can be rewritten as $$P(E|F^c) = \frac{P(E \cap F^c)}{P(F^c)} = \frac{P(E) - P(E\cap F)}{1-P(F)}$$

and since $$P(E \cap F)\approx .001026,$$ which can be shown be a simple extension of the mathematics in the first problem, we have $$P(E|F^c) = \frac{0.02255 - 0.001026}{1 - 0.02255} \approx 0.02202$$ 